## Experiments


In [12]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain.schema.output_parser import StrOutputParser 
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI

In [13]:
load_dotenv()

True

In [27]:
def load_txt(paths: list[str]) -> str:
    """Load the contents of a file and return it as a string"""
    for path in paths:
        with open(path, encoding="utf-8") as f:
            yield f.read()

def extract_variables(prompt: str, opening="{", closing="}") -> list[str]:
    """Extract and return variables encolsed in specific delimeters from the given prompt string."""
    variables = []
    inside_braces = False
    current_variable = []
    
    for char in prompt:
        if char == opening:
            inside_braces = True
            current_variable = []
            
        elif char == closing:
            if current_variable and inside_braces:
                variables.append(''.join(current_variable))
            inside_braces = False
        elif inside_braces:    
            current_variable.append(char)
            
    return variables

In [32]:
system_prompt, user_prompt = load_txt(["system_prompt.txt", "user_prompt.txt"])
user_prompt_template = ChatPromptTemplate.from_template(user_prompt)

print(f"Variables:\n\t SYSTEM: {extract_variables(system_prompt)}\n\t USER: {extract_variables(user_prompt)} ")

Variables:
	 SYSTEM: ['problem', 'time_allotted']
	 USER: ['user_input', 'code_snapshot', 'time_elapsed'] 


In [33]:
interviewer_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", user_prompt),
    ]
)

llm = ChatOpenAI(model="gpt-4o-mini")
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [34]:
interview_chain = interviewer_template | llm | StrOutputParser()

In [36]:
import time
from IPython.display import display, clear_output
import ipywidgets as widgets

# Provided variables
time_allotted = 40
user_input = None
code_snapshot = None
time_elapsed = None
i = 0

# Provided print statement
print("Welcome to the DSA Interview Simulator!")

# Function to handle interaction in Jupyter Notebook
def handle_interaction(submit_btn):
    global user_input, code_snapshot, time_elapsed, i
    
    # Update the time elapsed
    time_elapsed = i * 10

    # Capture user input and code snapshot from widgets
    user_input = user_input_widget.value
    code_snapshot = code_snapshot_widget.value

    # Prepare the variables dictionary as provided
    variables = {
        "problem": problem,
        "time_allotted": time_allotted,
        "user_input": user_input,
        "code_snapshot": code_snapshot,
        "time_elapsed": time_elapsed,
        "chat_history": memory.buffer_as_messages
    }

    # Invoke the AI chain with the provided variables
    ai = interview_chain.invoke(variables)
    
    # Display AI response
    clear_output(wait=True)
    print("Welcome to the DSA Interview Simulator!")
    print("AI:", ai)
    
    # Update widgets for the next input
    user_input_widget.value = ''
    code_snapshot_widget.value = ''

    # Simulate the memory updates and user input handling
    memory.chat_memory.add_ai_message(ai)
    memory.chat_memory.add_user_message(
        user_prompt_template.invoke({
            "user_input": user_input,
            "code_snapshot": code_snapshot,
            "time_elapsed": time_elapsed
        }).to_string()
    )
    
    # Increment the iteration counter
    i += 1
    
    # Display the interaction box again for further inputs
    display(interaction_box)

# Widgets for user input and code snapshot
user_input_widget = widgets.Textarea(
    value='',
    placeholder='Enter your message...',
    description='Message:',
    layout=widgets.Layout(width='100%', height='100px')
)

code_snapshot_widget = widgets.Textarea(
    value='',
    placeholder='Paste your code here...',
    description='Code:',
    layout=widgets.Layout(width='100%', height='200px')
)

# Submit button
submit_button = widgets.Button(
    description='Submit',
    button_style='primary',
    layout=widgets.Layout(width='100%')
)

# Attach the button click event to handle_interaction
submit_button.on_click(handle_interaction)

# Box layout for the interaction UI
interaction_box = widgets.VBox([user_input_widget, code_snapshot_widget, submit_button])

# Initial UI display
clear_output(wait=True)
print("Welcome to the DSA Interview Simulator!")

display(interaction_box)

Welcome to the DSA Interview Simulator!
AI: Thank you for your participation and for fixing the code! I'm glad to hear that you enjoyed the interview process.

### Final Thoughts:
You demonstrated a strong understanding of the problem and implemented an effective solution. Your willingness to adapt and fix issues during the interview is commendable and shows a good problem-solving mindset.

### Key Takeaways:
- You understood the requirements clearly and executed a well-structured approach.
- Your code is clean and functional.
- Keep practicing edge cases and handling various input scenarios to enhance your testing skills.

If you have any questions or need clarification on anything discussed, feel free to ask. 

Thank you again for your time, and I wish you the best in your future endeavors! Goodbye!
